In [11]:
import pandas as pd

In [12]:
df=pd.read_csv("fake_or_real_news.csv")

In [13]:
df.head()

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [14]:
df.describe()

,id
count,6335.000000
mean,5280.415627
std,3038.503953
min,2.000000
25%,2674.500000
50%,5271.000000
75%,7901.000000
max,10557.000000


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      6335 non-null   int64 
 1   title   6335 non-null   object
 2   text    6335 non-null   object
 3   label   6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [16]:
df.isnull().sum()

id       0
title    0
text     0
label    0
dtype: int64

In [17]:
df=df.fillna('')

In [18]:
df.isnull().sum()

id       0
title    0
text     0
label    0
dtype: int64

In [19]:
df.columns

Index(['id', 'title', 'text', 'label'], dtype='object')

In [20]:
df=df.drop(['id', 'title'], axis=1)

In [21]:
df.head()

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,It's primary day in New York and front-runners...,REAL


In [22]:
from nltk.corpus import stopwords

In [23]:
from nltk.stem.porter import PorterStemmer

In [24]:
import re

In [25]:
port_stem=PorterStemmer()

In [26]:
port_stem

<PorterStemmer>

In [27]:
port_stem.stem("Hi thIs is chando * % %@@@")

'hi this is chando * % %@@@'

In [28]:
def stemming(content):
    con=re.sub('[^a-zA-Z]', ' ', content)
    con=con.lower()
    con=con.split()
    con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    return con

In [ ]:
stemming('Hi this is sakshu')

'hi chando'

In [30]:
df['text']= df['text'].apply(stemming)

In [31]:
x=df['text']

In [32]:
y=df['label']

In [33]:
y.shape

(6335,)

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
x_train , x_test , y_train, y_test = train_test_split(x, y, test_size=0.20)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
vect=TfidfVectorizer()

In [38]:
x_train=vect.fit_transform(x_train)
x_test=vect.transform(x_test)

In [39]:
x_test.shape

(1267, 40002)

In [40]:
from sklearn.tree import DecisionTreeClassifier

In [41]:
model=DecisionTreeClassifier()

In [42]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [43]:
prediction=model.predict(x_test)

In [44]:
prediction

array(['FAKE', 'REAL', 'FAKE', ..., 'FAKE', 'REAL', 'FAKE'], dtype=object)

In [45]:
model.score(x_test, y_test)

0.813733228097869

In [46]:
import pickle

In [47]:
pickle.dump(vect, open('vector.pkl', 'wb'))

In [48]:
pickle.dump(model, open('model.pkl', 'wb'))

In [49]:
vector_form=pickle.load(open('vector.pkl', 'rb'))

In [50]:
load_model=pickle.load(open('model.pkl', 'rb'))

In [51]:
def fake_news(news):
    news=stemming(news)
    input_data=[news]
    vector_form1=vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return prediction

In [52]:
val=fake_news("""In these trying times, Jackie Mason is the Voice of Reason. [In this week’s exclusive clip for Breitbart News, Jackie discusses the looming threat of North Korea, and explains how President Donald Trump could win the support of the Hollywood left if the U. S. needs to strike first.  “If he decides to bomb them, the whole country will be behind him, because everybody will realize he had no choice and that was the only thing to do,” Jackie says. “Except the Hollywood left. They’ll get nauseous. ” “[Trump] could win the left over, they’ll fall in love with him in a minute. If he bombed them for a better reason,” Jackie explains. “Like if they have no transgender toilets. ” Jackie also says it’s no surprise that Hollywood celebrities didn’t support Trump’s strike on a Syrian airfield this month. “They were infuriated,” he says. “Because it might only save lives. That doesn’t mean anything to them. If it only saved the environment, or climate change! They’d be the happiest people in the world. ” Still, Jackie says he’s got nothing against Hollywood celebs. They’ve got a tough life in this country. Watch Jackie’s latest clip above.   Follow Daniel Nussbaum on Twitter: @dznussbaum """)

In [53]:
if val==[0]:
    print('reliable')
else:
    print('unreliable')

unreliable
